In [ ]:
import numpy as np
import os, sys
sys.path.append(os.getcwd()+"/..")
from rnn_scripts.train import *
from rnn_scripts.utils import *
from tasks.seqDS import *

import matplotlib.pyplot as plt
%matplotlib inline

from mayavi import mlab
mlab.init_notebook()

_,_,luts1,luts2 = np.load('../data/luts.npy')


In [ ]:
# Load models

fig_dir=os.getcwd()+"/../figures/"
model_dir = os.getcwd()+"/../models/"

model = "N512_T0222-185615"  #4 items

rnn,params,task_params,training_params = load_rnn(model_dir+model)


In [ ]:
# Some preprocessing / extracting parameters

dt =2
rnn.rnn.svd_orth()
set_dt(task_params,rnn,dt)
make_deterministic(task_params, rnn)
I,n,m,W = extract_loadings(rnn, orth_I=False,split=True)
alphas, I_orth = orthogonolise_Im(I,m)

In [ ]:
# Extract the trajectories

task_params["probe_dur"]=2
ks,phases,rates = get_traj(rnn,task_params,freq=8,amp_scale=1)

plt.figure(figsize=(4,4))
for i in range(len(ks)):
    plt.plot(ks[i,0], ks[i,1])
vm = np.max(ks)*1.1
plt.xlim(-vm,vm)
plt.ylim(-vm,vm)

In [ ]:
mlab.clf()
fig = mlab.figure(size = (1600,1600),\
            bgcolor = (1,1,1), fgcolor = (0.5, 0.5, 0.5))
luts = [luts1,luts2/3+luts1*2/3,luts2*2/3+luts1*1/3,luts2]
tw=.08
floor=-1.5
r_tor=1.5
r=2
r_s=1
cvs = np.sin((phases))

# plot floor
torus=def_torus(r_tor,r_s)
ks[:,:,-1]=ks[:,:,0]
mlab.mesh(torus[0], torus[1], np.zeros_like(torus[2])+floor
          , color=(0.5,0.5,0.5), opacity=0.05)

# plot trajectories
for i in range(len(ks)):
    x1,y1,z1 = tor(ks[i,1],ks[i,0], phases,r)
    surf1 = mlab.plot3d(x1, y1, z1,
                        cvs, tube_radius=tw, colormap='cool')

    surf1.module_manager.scalar_lut_manager.lut.table = luts[i]
    surf1.actor.property.lighting = False
   
    # plot shadows
    sh_surf1 = mlab.plot3d(x1,y1, floor*np.ones_like(phases),
                        cvs, tube_radius=tw, colormap='cool')

    shadow_lut1 = np.copy(luts[i])
    shadow_lut1[:,3]=20
    sh_surf1.module_manager.scalar_lut_manager.lut.table = shadow_lut1
    sh_surf1.actor.property.lighting = False

mlab.draw()
mlab.view(azimuth=8, elevation=67, distance=36, 
          focalpoint=np.array([ 0,  0, -3.5]))
mlab.plot3d(0,0,0)